# Purpose of this notebook
An obstacle for getting started with Neuroglancer is the data formats you can feed it are somewhat unfamiliar. For example, it does not accept TIFF format. It does this for a good reason though which is to be more efficient. One of the formats it accepts is called "precomputed" format, and that is the one we will use in this notebook. Fortunately there is a python pipeline for making precomputed data from TIFF files. 

Neuroglancer also is not set up to read CSV files. Instead, it uses the JSON file format, which essentially look like python dictionaries.

This notebook covers how to convert a custom annotated atlas volume (of the same format as the 'WHS_SD_rat_atlas_v3_annotation.tif' file on bucket) to precomputed format so that you can load it in to Neuroglancer. It also covers how to convert a CSV file formatted like the 'labels_v3.csv' file on bucket containing the region name mapping for the Rat MRI atlas into JSON format so that Neuroglancer can read it and display the region names in the bottom left corner when you hover over a region with your cursor.

In the following I use the original MRI atlas annotation volume and CSV file as an example. To use custom annotation volume and/or custom CSV file, replace the variables at the top of the notebook with your custom ones. The rest of the notebook should not (hopefully!) need to be changed before running.

## A quick note about Neuroglancer
Neuroglancer loads in datasets in "layers". A layer can be of type "image" (like what you would get as output from the light sheet microscope) or type "segmentation" (like an atlas annotation volume). The naming is a little confusing because both layer types refer to volumes (3-d objects). In this notebook, we are only concerned with a single layer: the annotation volume, which is a segmentation layer. If you were to make multiple annotation volumes (with different boundaries, etc.), each one of those would be a different layer. In Neuroglancer, you can overlay multiple layers or view them side-by-side. 

# Setup
In order to run the code in this notebook, you will need a conda environment with python3 and containing some additional libraries. This environment "ng_mriatlas" can be set up in the following way:
In terminal:
- conda create -n ng_mriatlas python=3.7.4 -y
- conda activate ng_mriatlas # (or source activate ng_mriatlas, depending on which version of conda you have)
- pip install cloud-volume
- pip install SimpleITK
- **pip install neuroglancer** <br>

\# To enable you to use jupyter notebooks to work with this environment as a kernel:
- pip install --user ipykernel
- python -m ipykernel install --user --name=ng_mriatlas

Once this is all installed, make sure to select this conda environment as the kernel when running this notebook (you might have to restart the notebook server)

In [ ]:
import os,csv,json
import numpy as np
from cloudvolume import CloudVolume
from cloudvolume.lib import mkdir, touch
import SimpleITK as sitk

from concurrent.futures import ProcessPoolExecutor

# Point to the custom annotation volume file that you have modified (in this example we point to the original one) 
custom_annotation_vol_path = 'WHS_SD_rat_atlas_v3_annotation.tif'

# Point to the csv file mapping region id to region name that you have modified (in this example we point to the original one)
annotation_csv_file = 'labels_v3.csv'

# Decide on a folder name where your layer for this volume is going to live. 
# IMPORTANT: For each custom atlas you need to change this so you don't 
# overwrite an old layer. You need the full path here. 
# In this example my layer folder will be 'my_custom_atlas1' saved in the current working directory
cwd = os.getcwd()
layer_dir = os.path.join(cwd,'my_custom_atlas1')
# Make the layer directory
if not os.path.exists(layer_dir):
    os.mkdir(layer_dir)
    print(f"created {layer_dir}")
    
# Finally, decide how many cpus you are willing and able to use for the parallelized conversion (see step 3)
cpus_to_use = 8

## Step 1: Convert the CSV annotation file into JSON file that Neuroglancer can read

In [ ]:
def make_annotation_json_file(csv_input_file):
    """ Take a csv file of the format of labels_v3.csv 
    containing the MRI atlas labels and converts it to a 
    JSON file so that Neuroglancer can read it in
    
    The JSON file will be saved with the same filename as 
    the CSV file but with .json extension instead of .csv
    """
    annotation_json_file = csv_input_file.replace('.csv','.json')

    with open(csv_input_file,'r') as infile:
        reader = csv.reader(infile)
        next(reader) # skips the header since we just want the data and we already know what the order of the columns is
        annotation_dict = {row[2]:row[1] for row in reader}
        annotation_dict_list = [{f"regionId":key,"regionName":f"{key}: {annotation_dict[key]}"} for key in annotation_dict.keys()]
    with open(annotation_json_file,'w') as outfile:
        outfile.write(
            '[' +
            ',\n'.join(json.dumps(i) for i in annotation_dict_list) +
            ']\n')
    # Now move the annotation file to the layer directory
    annotation_rel_path = annotation_json_file.split('/')[-1]
    dst = os.path.join(layer_dir,annotation_rel_path)
    os.rename(annotation_json_file,dst)
    print(f"Wrote file {dst}")
    return dst

In [ ]:
# run the function to convert to JSON
annotation_json_fullpath=make_annotation_json_file(annotation_csv_file)

## Step 2: Write the instructions ("info") file that will tell Neuroglancer about your annotation volume and the annotation label file you just created
The info file is a required file for the precomputed data format. It is a JSON file containing things like the shape and physical resolution of your volume, and it will also be where we tell it to read in our labels for the different anatomical regions

In [ ]:
def make_info_file(resolution_xyz,volume_size_xyz,annotation_filename,layer_dir):
    """ Make an JSON-formatted file called the "info" file
    for use with the precomputed data format. 
    Precomputed is one of the formats that Neuroglancer can read in.  
    --- parameters ---
    resolution_xyz:      A tuple representing the size of the pixels (dx,dy,dz) 
                         in nanometers, e.g. (20000,20000,5000) for 20 micron x 20 micron x 5 micron
    
    volume_size_xyz:     A tuple representing the number of pixels in each dimension (Nx,Ny,Nz)
    
    annotation_filename: The relative path (from layer_dir) to the JSON-formatted file
                         that we created above containing the labels of the different regions.
                         
    layer_dir:           The directory where the precomputed data will be
                         saved
    """
    info = CloudVolume.create_new_info(
        num_channels = 1,
        layer_type = 'segmentation', # 'image' or 'segmentation'
        data_type = 'uint16', # 32 not necessary for atlases unless you have > 2^(32)-1 labels  
        encoding = 'raw', # other options: 'jpeg', 'compressed_segmentation' (req. uint32 or uint64)
        resolution = resolution_xyz, # X,Y,Z values in nanometers, 40 microns in each dim
        voxel_offset = [ 0, 0, 0 ], # values X,Y,Z values in voxels
        chunk_size = [ 1024, 1024, 1 ], # rechunk of image X,Y,Z in voxels.
        volume_size = volume_size_xyz, # X,Y,Z size in voxels
    )

    vol = CloudVolume(f'file://{layer_dir}', info=info)
    vol.provenance.description = "A test info file" # can change this if you want a description
    vol.provenance.owners = [''] # list of contact email addresses
    # Saves the info and provenance files for the first time
    vol.commit_info() # generates file://bucket/dataset/layer/info json file
    vol.commit_provenance() # generates file://bucket/dataset/layer/provenance json file
    # add a key for the custom atlas annotations to the info file and overwrite it 
    info_dict = vol.info
    info_dict['atlas_path'] = annotation_filename
    info_filename = '/'.join(vol.info_cloudpath.split('/')[2:]) 
    with open(info_filename,'w') as outfile:
        json.dump(info_dict,outfile,sort_keys=True,indent=2)
    print("Created CloudVolume info file: ",vol.info_cloudpath)

    return vol

In [ ]:
## Make the info file

# The MRI atlas has 39 micron isotropic resolution - we need nanometers though
resolution_xyz = (39000,39000,39000)
# Load the MRI annotation volume into memory and get its shape 
annotation_vol = np.array(sitk.GetArrayFromImage(
    sitk.ReadImage(custom_annotation_vol_path)),dtype=np.uint16,order='F')
z_dim,y_dim,x_dim = annotation_vol.shape
volume_size_xyz = (x_dim,y_dim,z_dim)

# Now get the relative path to the annotation file
annotation_json_relpath = annotation_json_fullpath.split('/')[-1]

# Write the info file
vol = make_info_file(
    resolution_xyz=resolution_xyz,
    volume_size_xyz=volume_size_xyz,
    annotation_filename=annotation_json_relpath,
    layer_dir=layer_dir)


# Step 3: Convert annotation volume to precomputed data format
First we create a directory (the "progress_dir") at the same folder level as the layer directory to keep track of the progress of the conversion. 
All the conversion does is copy the numpy array representing the 3d volume to a new object "vol". This is done one plane at a time (although it is parallelized). As each plane is converted, an empty file is created in the progress_dir with the name of the plane. By the end of the conversion, there should be as many files in this progress_dir as there are z planes. 

In [ ]:
layer_name = layer_dir.split('/')[-1]
parent_dir = '/'.join(layer_dir.split('/')[:-1])
# print(parent_dir)
progress_dir = mkdir(parent_dir+ f'/progress_{layer_name}') # unlike os.mkdir doesn't crash on prexisting 
print(f"created directory: {progress_dir}")

In [ ]:
def process_slice(z):
    """ This function copies a 2d image slice from the atlas volume
    to the cloudvolume object, vol. We will run this in parallel over 
    all z planes
    ---parameters---
    z:    An integer representing the 0-indexed z plane to be converted
    """
    if os.path.exists(os.path.join(progress_dir, str(z))):
        print(f"Slice {z} already processed, skipping ")
        return
    if z >= z_dim: # z is zero indexed and runs from 0-(z_dim-1)
        print("Index {z} >= z_dim of volume, skipping")
        return
    print('Processing slice z=',z)
    array = annotation_vol[z].reshape((1,y_dim,x_dim)).T
    vol[:,:, z] = array
    touch(os.path.join(progress_dir, str(z)))
    return "success"


In [ ]:
# Run the conversion in parallel. It's not a huge amount of processing but the more cores the better

# First figure out if there are any planes that have already been converted 
# by checking the progress dir
done_files = set([ int(z) for z in os.listdir(progress_dir) ])
all_files = set(range(vol.bounds.minpt.z, vol.bounds.maxpt.z))
# Figure out the ones we still need to convert 
to_upload = [ int(z) for z in list(all_files.difference(done_files)) ]
to_upload.sort()
print(f"Have {len(to_upload)} planes to upload")
with ProcessPoolExecutor(max_workers=cpus_to_use) as executor:
    for result in executor.map(process_slice,to_upload):
        try:
            print(result)
        except Exception as exc:
            print(f'generated an exception: {exc}')

# Step 4: Host the precomputed data on your machine so that Neuroglancer can see it
This step is really easy! Note: Exectuing the code below will cause your jupyter notebook to hang, so it is better to run the following code in a new ipython terminal (make sure to have the ng_mriatlas conda environment activated in that python session) rather than the notebook. 

```python
from cloudvolume import CloudVolume
vol = CloudVolume(f'file://{layer_dir}')
vol.viewer(port=1338)
```

# Step 5: View your custom volume and labels in Neuroglancer
Step 4 hosts your data via http on port 1338 of your local machine. To actually view your data in Neuroglancer, there are two ways to do this. You can either load the data in manually in the browser or load it in with python. 

For the manual method, open up the Braincogs Neuroglancer client: [https://nglancer.pni.princeton.edu](https://nglancer.pni.princeton.edu) (you must be using a Princeton VPN) and then click the "+" in the upper left hand corner of the screen once the black screen loads. To load in your data, type the following into the source text box:<br>
> precomputed://http://localhost:1338 <br>

Then hit tab and name your layer if you'd like. Hit enter or the "add layer" button and your layer should load into Neuroglancer. Hopefully the labels you added should be showing up in the bottom left when you hover over a region. 

For the python method, you can do this by executing the following cell. Make sure you have hosted the data in another python instance somewhere on your local machine at port 1338.

In [ ]:
import neuroglancer

# Set which client you want to use - use the BRAINCOGS client to get the latest features 
neuroglancer.set_static_content_source(url='https://nglancer.pni.princeton.edu')

# Make a viewer object that represents your connection to the Neuroglancer window
viewer = neuroglancer.Viewer()

# Load in the layer with name "my_custom_atlas"
with viewer.txn() as s:
    s.layers['my_custom_atlas'] = neuroglancer.SegmentationLayer(source='precomputed://http://localhost:1338',
    )
print(viewer)